Importation des librairies

In [84]:
# import libraries
from IPython.display import Image, display
import numpy as np
import os, sys, tarfile
from os.path import join
from PIL import ImageFile
import pandas as pd
from matplotlib import cm
import seaborn as sns
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50, vgg16
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import mean_squared_error, mean_absolute_error, roc_auc_score, classification_report, confusion_matrix
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
from sklearn import svm
from sklearn.mixture import GaussianMixture
from sklearn.isotonic import IsotonicRegression
import re
import scipy.io
import matplotlib.pyplot as plt
from datetime import datetime, date, time
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import layers
from tensorflow.keras import Model
from keras_preprocessing.image import ImageDataGenerator

Récupération du dataset

In [ ]:
!wget http://ai.stanford.edu/~jkrause/car196/car_ims.tgz ./car_ims.tgz

--2022-07-29 07:46:30--  http://ai.stanford.edu/~jkrause/car196/car_ims.tgz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1956628579 (1.8G) [application/x-gzip]
Saving to: ‘car_ims.tgz’

car_ims.tgz         100%[===================>]   1.82G  21.8MB/s    in 87s     

2022-07-29 07:47:57 (21.5 MB/s) - ‘car_ims.tgz’ saved [1956628579/1956628579]

/content/sample_data/car_ims.tgz: Scheme missing.
FINISHED --2022-07-29 07:47:57--
Total wall clock time: 1m 27s
Downloaded: 1 files, 1.8G in 1m 27s (21.5 MB/s)


Extraction du dossier

In [ ]:
def extract(tar_url, extract_path='./data'):
    tar = tarfile.open(tar_url, 'r')
    for item in tar:
        tar.extract(item, extract_path)
        if item.name.find(".tgz") != -1 or item.name.find(".tar") != -1:
            extract(item.name, "./" + item.name[:item.name.rfind('/')])
try:

    extract('./car_ims.tgz')
    print ('Done.')
except:
    name = os.path.basename(sys.argv[0])
    print (name[:name.rfind('.')], '<filename>')

ipykernel_launcher <filename>


Récupération des data d'entrainement

In [ ]:
# recup all data for the training set
local_folder_train = './data/car_ims/'
train_img_paths_n = [join(local_folder_train,filename) for filename in os.listdir(local_folder_train)]
print(len(train_img_paths_n))
# pre_trained_model = InceptionV3(
#     input_shape=(150, 150, 3), include_top=False, weights=None)
# pre_trained_model.load_weights(local_weights_file)


16185


In [ ]:
#import mat table
mat = scipy.io.loadmat('./cars_annos.mat')
annot = mat['annotations']
annotd = annot.dtype
classn = mat['class_names']
classnd = classn.dtype
annotation = {n: annot[n][0, 0] for n in annotd.names}
print(len(classn[0]))


196


In [ ]:
# transform mat table to dict array
j = 0
for k, v in annotation.items() :
    value = []
    for a in annot :
        for i in range(0,len(a)) :
            if j >= 1 :
                value.append(a[i][j][0][0])
            else :
                value.append(a[i][j][0])
    annotation[k] = value
    j += 1
annotation

In [ ]:
# df_annot
columns = [n for n in annotation.keys()]
df_annot = pd.DataFrame(annotation, columns=columns)
df_annot

,relative_im_path,bbox_x1,bbox_y1,bbox_x2,bbox_y2,class,test
0,car_ims/000001.jpg,112,7,853,717,1,0
1,car_ims/000002.jpg,48,24,441,202,1,0
2,car_ims/000003.jpg,7,4,277,180,1,0
3,car_ims/000004.jpg,33,50,197,150,1,0
4,car_ims/000005.jpg,5,8,83,58,1,0
...,...,...,...,...,...,...,...
16180,car_ims/016181.jpg,38,36,375,234,-60,1
16181,car_ims/016182.jpg,29,34,235,164,-60,1
16182,car_ims/016183.jpg,25,32,587,359,-60,1
16183,car_ims/016184.jpg,56,60,208,186,-60,1


In [ ]:
#df for class_names
class_names = []
for i in range(0,len(classn[0])) :
    class_names.append(classn[0][i][0])
c = np.array(class_names)
columns = ['class']
df_class = pd.DataFrame()
df_class['class'] = c.tolist()
df_class

,class
0,AM General Hummer SUV 2000
1,Acura RL Sedan 2012
2,Acura TL Sedan 2012
3,Acura TL Type-S 2008
4,Acura TSX Sedan 2012
...,...
191,Volkswagen Beetle Hatchback 2012
192,Volvo C30 Hatchback 2012
193,Volvo 240 Sedan 1993
194,Volvo XC90 SUV 2007


In [ ]:
# df en deux suivant column test
df_train = pd.DataFrame()
#print(df_annot)
df_train['img'] = df_annot.relative_im_path
df_train['class'] = df_annot['class']
selRows = df_train[df_train['class'] <= -60].index
df_train = df_train.drop(selRows, axis=0)
for i in range(0,196) :
    a= i+1
    df_train['class'] = df_train['class'].replace({a:df_class['class'][i]})
df_train

,img,class
0,car_ims/000001.jpg,AM General Hummer SUV 2000
1,car_ims/000002.jpg,AM General Hummer SUV 2000
2,car_ims/000003.jpg,AM General Hummer SUV 2000
3,car_ims/000004.jpg,AM General Hummer SUV 2000
4,car_ims/000005.jpg,AM General Hummer SUV 2000
...,...,...
10512,car_ims/010513.jpg,Honda Odyssey Minivan 2007
10513,car_ims/010514.jpg,Honda Odyssey Minivan 2007
10514,car_ims/010515.jpg,Honda Odyssey Minivan 2007
10515,car_ims/010516.jpg,Honda Odyssey Minivan 2007


In [ ]:
df_test = pd.DataFrame()
df_test['img'] = df_annot.relative_im_path[df_annot.test == 1]
df_test['class'] = df_annot['class'][df_annot.test == 1]
for i in range(0,196) :
    a= i+1
    df_test['class'] = df_test['class'].replace({a:df_class['class'][i]})
df_test

In [ ]:
df_number = pd.DataFrame(df_train.groupby(['class']).count())
print(df_number)
df_number.plot(rot=45)


In [ ]:
# image data generator : .1/255
datagen=ImageDataGenerator(rescale=1./255, validation_split=0.2)
datagen


In [ ]:
train_generator=datagen.flow_from_dataframe(
    dataframe=df_train,
    directory="/content/sample_data/data/",
    x_col="img",
    y_col="class",
    subset="training",
    target_size=(150, 150),
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical")
    
valid_generator=datagen.flow_from_dataframe(
    dataframe=df_train,
    directory="/content/sample_data/data/",
    x_col="img",
    y_col="class",
    subset="validation",
    target_size=(150, 150),
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical")

# test_datagen=ImageDataGenerator(rescale=1./255.)
# test_generator=test_datagen.flow_from_dataframe(
#     dataframe=df_test,
#     directory="./data/",
#     x_col="img",
#     y_col=None,
#     batch_size=32,
#     seed=42,
#     shuffle=False,
#     class_mode=None)

Found 8414 validated image filenames belonging to 127 classes.
Found 2103 validated image filenames belonging to 127 classes.


In [ ]:
# Our input feature map is 150x150x3: 150x150 for the image pixels, and 3 for
# the three color channels: R, G, and B
img_input = layers.Input(shape=(150, 150, 3))

# First convolution extracts 16 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(16, 3, activation='relu')(img_input)
x = layers.MaxPooling2D(2)(x)

# Second convolution extracts 32 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

# Third convolution extracts 64 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

In [ ]:
# Flatten feature map to a 1-dim tensor so we can add fully connected layers
x = layers.Flatten()(x)

# Create a fully connected layer with ReLU activation and 512 hidden units
x = layers.Dense(512, activation='relu')(x)

# Create output layer with a single node and sigmoid activation
output = layers.Dense(127, activation='softmax')(x)

# Create model:
# input = input feature map
# output = input feature map + stacked convolution/maxpooling layers + fully 
# connected layer + sigmoid output layer
model = Model(img_input, output)


In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
# STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit(train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10
)

In [ ]:
img_path = '/content/img.jpg'
img = image.load_img(img_path, target_size=(150, 150))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
preds = model.predict(x)

# print('Predicted:', decode_predictions(preds, top=3)[0])

In [ ]:
train_generator=datagen.flow_from_dataframe(
    dataframe=df_train,
    directory="/content/sample_data/data/",
    x_col="img",
    y_col="class",
    subset="training",
    target_size=(224, 224),
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical")
    
valid_generator=datagen.flow_from_dataframe(
    dataframe=df_train,
    directory="/content/sample_data/data/",
    x_col="img",
    y_col="class",
    subset="validation",
    target_size=(224, 224),
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical")
model2 = ResNet50V2()
model2.summary()

In [ ]:
datagen

In [127]:
cnn = Sequential()
# cnn.get_config()
for layer in model.layers[:-1]:
    cnn.add(layer)
# cnn.summary()

In [128]:
for layer in cnn.layers:
    layer.trainable = False
cnn.add(Dense(units=127, activation='softmax'))


In [129]:
cnn.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
cnn.summary()

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
# STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
cnn.fit(train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=2
)

In [ ]:
image = tf.io.read_file('/content/img.jpg')
image = tf.image.decode_jpeg(image)
image = tf.cast(image, tf.float32)
image = tf.image.resize_with_crop_or_pad(
    image, target_height=224, target_width=224)
image = preprocess_input(image)
predictions = cnn.predict(image)
